## Advanced Lane Finding Project

The goals / steps of this project are the following:

* Compute the camera calibration matrix and distortion coefficients given a set of chessboard images.
* Apply a distortion correction to raw images.
* Use color transforms, gradients, etc., to create a thresholded binary image.
* Apply a perspective transform to rectify binary image ("birds-eye view").
* Detect lane pixels and fit to find the lane boundary.
* Determine the curvature of the lane and vehicle position with respect to center.
* Warp the detected lane boundaries back onto the original image.
* Output visual display of the lane boundaries and numerical estimation of lane curvature and vehicle position.

---

# IMPORT PACKAGES

In [1]:
import numpy as np
import cv2
import glob
import pickle
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
%matplotlib inline

#

##  CALIBRATE CAMERA

### Find Chessboard corners from sample images

In [2]:

# prepare object points, like (0,0,0), (1,0,0), (2,0,0) ....,(6,5,0)
objp = np.zeros((6*9,3), np.float32)
objp[:,:2] = np.mgrid[0:9, 0:6].T.reshape(-1,2)

# Arrays to store object points and image points from all the images.
objpoints = [] # 3d points in real world space
imgpoints = [] # 2d points in image plane.

# Make a list of calibration images
images = glob.glob('camera_cal/calibration*.jpg')

# Step through the list and search for chessboard corners
for image in images:
    
    # Read Image
    img = mpimg.imread(image)
    
    # Convert to Gray Scale
    gray = cv2.cvtColor(img, cv2.COLOR_RGB2GRAY)

    # Find the chessboard corners
    ret, corners = cv2.findChessboardCorners(gray, (9,6), None)

    # If found, add object points, image points
    if ret == True:
        objpoints.append(objp)
        imgpoints.append(corners)
        

### Compute and Save Camera Matrix, Distortion

In [3]:
# Do camera calibration given object points and image points
ret, mtx, dist, rvecs, tvecs = cv2.calibrateCamera(objpoints, imgpoints, img.T.shape[1:],None, None)

# Save the camera calibration result for later use
# Ignore rvecs / tvecs for now
dist_pickle = {}
dist_pickle["mtx"] = mtx
dist_pickle["dist"] = dist
pickle.dump( dist_pickle, open( "mtx_dist_pickle.p", "wb" ) )


## LANE FINDING PIPELINE

#### UNDISTORT IMAGE

In [4]:
def undistort(img):
    # pick camera calibartion and distortion matrices from global variables
    global mtx, dist, cnt
    
    # undistort image
    undistort_img = cv2.undistort(img, mtx, dist, None, mtx)
    
    # return undistorted image
    return(undistort_img)

#### PRESPECTIVE TRANSFORM

In [5]:

# generate perspective transform matrix
def gen_transform_matrix():
    src = np.float32(
             [[582, 455],
             [700, 455],
             [1150, 720],
             [150, 720]])

    dst = np.float32(
             [[300, 0],
             [1000, 0],
             [1000, 720],
             [300, 720]])

    # perspective transform
    M = cv2.getPerspectiveTransform(src,dst)

    # Inverse perspective transform
    Minv = cv2.getPerspectiveTransform(dst,src)

    return(M, Minv)

M, Minv = gen_transform_matrix()

# warp image by perspective transform
def warp(img,M):
    
    # perspective transform image
    if len(img.shape) > 2:
        warped = cv2.warpPerspective(img, M, img.shape[:2][::-1], flags=cv2.INTER_LINEAR)
    else:
        warped = cv2.warpPerspective(img, M, img.shape[::-1], flags=cv2.INTER_LINEAR)
    
    return(warped)


#### THRESHOLDS

In [6]:
# Define a function that applies Sobel x or y, 
# then takes an absolute value and applies a threshold.
def abs_sobel_thresh(gray_img, orient='x', sobel_kernel=3, thresh=(0,255)):
    
        # Take derivative in x or y give orient = 'x' or 'y'
    if orient=='x':
        sobel_drv = cv2.Sobel(gray_img, cv2.CV_64F, 1, 0, ksize=sobel_kernel)
    elif orient=='y':
        sobel_drv = cv2.Sobel(gray_img, cv2.CV_64F, 0, 1, ksize=sobel_kernel)
    else:
        print("Invalid Orient, provide orient either x or y")
    
    # Take absolute value of the derivative or gradient
    abs_sobel_drv = np.absolute(sobel_drv)
    
    # Scale to 8-bit (0-255) then convert to type=np.uint8
    scale_sobel = np.uint8(255*abs_sobel_drv/np.max(abs_sobel_drv))
    
    # Create a mask of 1's where the scaled gradient magnitude
    #     is > thresh_min and < thresh_max
    binary_output = np.zeros_like(scale_sobel)
    binary_output[ (scale_sobel>thresh[0]) & (scale_sobel<=thresh[1])] = 1 
    
    return binary_output
    


#### BINARY IMAGE FROM COLOR AND GRADIENT

In [7]:

def binary_image(img):
    
    # Convert RGB to HLS,HSV and seperate S-channel from HLS and v-channel from HSV
    # s-channel for color and v-channel for grient
    # v-channel shows slightly stronger gradient for high contrast background
    # color filtered v-channel used to negate noise from interlaced shadows
    s_channel = cv2.cvtColor(img, cv2.COLOR_RGB2HLS)[:,:,2]
    v_channel = cv2.cvtColor(img, cv2.COLOR_RGB2HSV)[:,:,2]
    
    
    # ***** Color Filtering on S & V Channels ******
    #  low-threshold for v-channel is 1.5*(s-channel low-threshold)
    
    # Color filtered s-channel binary image
    binary_s_channel = np.zeros_like(s_channel)
    binary_s_channel[ (s_channel > 90) & (s_channel <=255) ] = 1
    
    # Color filtered v-channel binary image
    binary_v_channel = np.zeros_like(v_channel)
    binary_v_channel[ (v_channel > 170) & (v_channel <=255) ] = 1
    
    
    # ***** Compute V-Channel Gradient *****
    
    # Choose a Sobel kernel size
    ksize = 7 # Choose a larger odd number to smooth gradient measurements

    # Gradient in x-direction ( return binary image)
    gradx = abs_sobel_thresh(v_channel, orient='x', sobel_kernel=ksize, thresh=(20, 150))
    
    # Gradient Direction ( return binary image)
    # dir_binary = dir_threshold(v_channel, sobel_kernel=ksize, thresh=(0.75, 1.25))

    # ***** Combine Color Filtering and Gradient Filtering *****
    combined = np.zeros_like(v_channel)
    
    #  Gradient direction preserves only line edges while eating the fill,
    #  which makes senses as line fill shows orthogonal gradient
    # combined[ (((binary_s_channel == 1) & (binary_v_channel == 1)) | (gradx == 1)) & (dir_binary==1)  ] = 255 # gray scale
    
    # x-gradient adds white lines to  lines from color channels
    # s-channel preserves left yellow line, while v-channel filters shadows.
    combined[ ((binary_s_channel == 1) & (binary_v_channel == 1)) | ( gradx == 1 ) ] = 1 # gray scale
    
    # ***** Return Binary Image *****
    return(combined)


#### FINDING LANE LINES

#### Apply Mask

In [8]:
def region_of_interest(img, vertices):
    """
    Applies an image mask.
    
    Only keeps the region of the image defined by the polygon
    formed from `vertices`. The rest of the image is set to black.
    """
    #defining a blank mask to start with
    mask = np.zeros_like(img)   
    
    #defining a 3 channel or 1 channel color to fill the mask with depending on the input image
    if len(img.shape) > 2:
        channel_count = img.shape[2]  # i.e. 3 or 4 depending on your image
        ignore_mask_color = (255,) * channel_count
    else:
        ignore_mask_color = 255
        
    #filling pixels inside the polygon defined by "vertices" with the fill color    
    cv2.fillPoly(mask, vertices, ignore_mask_color)
    
    #returning the image only where mask pixels are nonzero
    masked_image = cv2.bitwise_and(img, mask)
    return masked_image


# Mask Non-Lane Lines

def apply_mask(image):
    ## Parameters for region of interest
    ## will define trapeziod to filter Lane lines
    width  = image.shape[1]   # Image Widht
    height = image.shape[0]   # Image height
    ll = (0, height)       # Lower Left Vertex
    lr = (width, height)      # Lower Right Vertex
    tl = (-50+width/2, 80+height/2)      # Top Left Vertex
    tr = (50+width/2, 80+height/2)      # Top Right Vertex
    vertices = np.array( [[ ll, tl, tr, lr]], dtype=np.int32)

    fimage = region_of_interest(image, vertices )
    return(fimage)

  

#### FINDING LANE PIXELS

In [9]:

def window_mask(width, height, img_ref, center,level):
    output = np.zeros_like(img_ref)
    output[int(img_ref.shape[0]-(level+1)*height):int(img_ref.shape[0]-level*height),max(0,int(center-width/2)):min(int(center+width/2),img_ref.shape[1])] = 1
    return output

def find_window_centroids(warped, window_width, window_height, margin, minpix):
    
    window_centroids = [] # Store the (left,right) window centroid positions per level
    window = np.ones(window_width) # Create our window template that we will use for convolutions
    
    # First find the two starting positions for the left and right lane by using 
    # np.sum to get the vertical image slice
    # and then np.convolve the vertical image slice with the window template 
    
    # Sum quarter bottom of image to get slice, could use a different ratio
    l_sum = np.sum(warped[int(3*warped.shape[0]/4):,:int(warped.shape[1]/2)], axis=0)   
    l_center = np.argmax(np.convolve(window,l_sum))-window_width/2
    r_sum = np.sum(warped[int(3*warped.shape[0]/4):,int(warped.shape[1]/2):], axis=0)
    r_center = np.argmax(np.convolve(window,r_sum))-window_width/2+int(warped.shape[1]/2)
    
    # Add what we found for the first layer
    window_centroids.append((l_center,r_center))
    
    # Go through each layer looking for max pixel locations
    for level in range(1,(int)(warped.shape[0]/window_height)):
        
        # convolve the window into the vertical slice of the image
        image_layer = np.sum(warped[int(warped.shape[0]-(level+1)*window_height):int(warped.shape[0]-level*window_height),:], axis=0)
        conv_signal = np.convolve(window, image_layer)
        
        
        # Find the best left centroid by using past left center as a reference
        # Use window_width/2 as offset because convolution signal reference is 
        # at right side of window, not center of window
        offset = window_width/2
        l_min_index = int(max(l_center+offset-margin,0))
        l_max_index = int(min(l_center+offset+margin,warped.shape[1]))
        if np.sum(conv_signal[l_min_index:l_max_index]) > minpix:   # No Update to centroid if lane not found
            l_center = np.argmax(conv_signal[l_min_index:l_max_index])+l_min_index-offset
        
        # Find the best right centroid by using past right center as a reference
        r_min_index = int(max(r_center+offset-margin,0))
        r_max_index = int(min(r_center+offset+margin,warped.shape[1])) 
        if np.sum(conv_signal[r_min_index:r_max_index]) > minpix :   # No update to centriod if lane not found
            r_center = np.argmax(conv_signal[r_min_index:r_max_index])+r_min_index-offset
        
        # Add what we found for that layer
        # print(np.sum(conv_signal[l_min_index:l_max_index]), np.sum(conv_signal[r_min_index:r_max_index]))
        window_centroids.append((l_center,r_center))

    return window_centroids



In [10]:

def sliding_search(warped):

    # window settings
    window_width = 50 
    window_height = 80 # Break image into 9 vertical layers since image height is 720
    margin = 100 # How much to slide left and right for searching
    minpix = 15000  # Min pixels to be found to update search window centroid

    window_centroids = find_window_centroids(warped, window_width, window_height, margin, minpix)

    # If we found any window centers
    if len(window_centroids) > 0:
    
        # Points used to draw all the left and right windows
        l_points = np.zeros_like(warped)
        r_points = np.zeros_like(warped)
    
        # Go through each level and draw the windows 	
        for level in range(0,len(window_centroids)):
            
            # Window_mask is a function to draw window areas
            l_mask = window_mask(window_width,window_height,warped,window_centroids[level][0],level)
            r_mask = window_mask(window_width,window_height,warped,window_centroids[level][1],level)
            
            # Add graphic points from window mask here to total pixels found 
            l_points[(l_points == 255) | ((l_mask == 1) ) ] = 255
            r_points[(r_points == 255) | ((r_mask == 1) ) ] = 255
    
        # Draw the results
        template = np.array(r_points+l_points,np.uint8) # add both left and right window pixels together
        zero_channel = np.zeros_like(template) # create a zero color channel
        template = np.array(cv2.merge((zero_channel,template,zero_channel)),np.uint8) # make window pixels green
        warped[warped==1]=255
        warpage = np.array(cv2.merge((warped,warped,warped)),np.uint8) # making the original road pixels 3 color channels
        output = cv2.addWeighted(warpage, 1, template, 0.5, 0.0) # overlay the orignal road image with window results
 
    # If no window centers found, just display orginal road image
    else:
        output = np.array(cv2.merge((warped,warped,warped)),np.uint8)

    return(output)

# Display the final results
#plt.imshow(output)
#plt.title('window fitting results')
#plt.show()


In [11]:



def adjust_lane_fit(warped,lane_fit):
    
    new_lane_fit =()  # new polynomial fit, 0:left lane, 1:right lane
    nonzero = warped.nonzero()
    nonzeroy = np.array(nonzero[0])
    nonzerox = np.array(nonzero[1])
    margin = 100
    not_enough_points = 0  # check for enough points in search zone
                           # would trigger exhaustive lane search
    
    for _fit in lane_fit:
        
        lane_inds = ((nonzerox > (_fit[0]*(nonzeroy**2) + _fit[1]*nonzeroy + _fit[2] - margin)) & (nonzerox < (_fit[0]*(nonzeroy**2) + _fit[1]*nonzeroy + _fit[2] + margin))) 
        
        # print(lane_inds.shape)
        
        # check for enough indices
        if len(lane_inds) < 100:
            not_enough_points = 1
        
        # Again, extract left and right line pixel positions
        x = nonzerox[lane_inds]
        y = nonzeroy[lane_inds] 
        
        # Fit a second order polynomial to each
        fit = np.polyfit(y, x, 2)
        
        new_lane_fit = new_lane_fit + (fit,)
        
    # find lane fit from exhaustive search, when not enough points
    #  found with adaptive lane line search
    if not_enough_points:
        print("===== Exhaustive Search Triggered ====")
        new_lane_fit = get_lane_fit(warped)
    
    return(new_lane_fit)


def get_lane_fit(warped):
    
    # window settings
    window_width = 50 
    window_height = 80 # Break image into 9 vertical layers since image height is 720
    margin = 100       # How much to slide left and right for searching
    minpix = 15000     # Min pixels to be found to update search window centroid
    lane_fit  = []
    
    # 
    window_centroids = find_window_centroids(warped, window_width, window_height, margin, minpix)

    l_points = []
    r_points = []
    
    # Go through each level and draw the windows 	
    for level in range(0,len(window_centroids)):
            
        # collate window centriod points
        l_points.append(window_centroids[level][0])
        r_points.append(window_centroids[level][1])
    
    # Lane pixels with in windows
    y  = range((int(window_height/2)),warped.shape[0],window_height)[::-1]

               
    lane_fit = (
                np.polyfit(y, l_points, 2),
                np.polyfit(y, r_points, 2)
                )
    
    return(lane_fit)



def _draw_lane(warped, lane_fit):
    # Takes in binary image and lane_fit
    # return color image with green lane marking
    
    left  = lane_fit[0]  # left  lane coeffs
    right = lane_fit[1]  # right lane coeffs
         
    lane_mask = np.zeros_like(warped)
    lane_mask1 = np.zeros_like(warped)
    
    ycord = np.arange(0,warped.shape[0])
    xcord = np.arange(0,warped.shape[1])
    
    left_inds = np.array(left[0]*(ycord**2) + left[1]*ycord + left[2],np.int16)
    right_inds = np.array(right[0]*(ycord**2) + right[1]*ycord + right[2],np.int16)
    
    lane_mask[ ycord, left_inds] = 255
    lane_mask1[ ycord, right_inds] = 255
    
    output = np.array(cv2.merge((lane_mask,warped*255,lane_mask1)),np.uint8)
    
    # output = np.array(cv2.merge((lane_mask,lane_mask,lane_mask)),np.uint8)
    
    return(output)

def draw_lane(warped, lane_fit):

    # Create an image to draw the lines on
    warp_zero = np.zeros_like(warped).astype(np.uint8)
    color_warp = np.dstack((warp_zero, warp_zero, warp_zero))

    # compute x and y points for left and right lane
    left = lane_fit[0]
    right = lane_fit[1]
    ploty = np.arange(0,warped.shape[0])
    leftx = np.array(left[0]*(ploty**2) + left[1]*ploty + left[2],np.int16)
    rightx = np.array(right[0]*(ploty**2) + right[1]*ploty + right[2],np.int16)
    
    
    # Recast the x and y points into usable format for cv2.fillPoly()
    pts_left = np.array([np.transpose(np.vstack([rightx, ploty]))])   # left lane
    pts_right = np.array([np.flipud(np.transpose(np.vstack([leftx, ploty])))])  # right lane
    pts = np.hstack((pts_left, pts_right))

    # Draw the lane onto the warped blank image
    cv2.fillPoly(color_warp, np.int_([pts]), (0,255, 0))

    return(color_warp)


def mark_lane(warped):
    
    # to identify first image
    # defined later
    global lane_fit;
    
    
    # compute lane coeffs for first image
    if 'lane_fit' not in globals():
        lane_fit = get_lane_fit(warped)  # return coeffs from centriods
        
    # adjust lane coeffs 
    lane_fit = adjust_lane_fit(warped,lane_fit)
    
    # draw lane marking
    lane_img = draw_lane(warped,lane_fit)  # binary_img in & color_img out
    
    return(lane_img)
    


## RUN PIPELINE ON VIDEO

In [12]:

# Import everything needed to edit/save/watch video clips
from moviepy.editor import VideoFileClip
from IPython.display import HTML


In [13]:

def process_image(img):
    # NOTE: The output you return should be a color image (3 channel) for processing video below
    
    global cnt, M, Minv
    cnt += 1 
    
    # Pipeline goes here
    
    # Undistort image
    undist_img = undistort(img)             # color_img in & out
    
    # Generate binary image from color and gradient
    binary_img = binary_image(undist_img)   # color_img in & binary_img out
    
    # Apply Mask
    masked_img = apply_mask(binary_img)      # binary_img in & binary_img out
    
    # generate transform matrix
    # M, Minv = gen_transform_matrix()          
    
    # Transform masked image 
    warp_img = warp(masked_img,M)          # binary_img in & binary_img out 
    
    # Mark Lane with green color
    lane_img = mark_lane(warp_img)          # binary_img in & color_img out
    
    # Unwarp lane image
    unwarp_lane_img = warp(lane_img,Minv)   # color_img in & color_img out
    
    
    # Merge Lane image on to original image
    output = cv2.addWeighted(img, 1, unwarp_lane_img, 0.5, 0.0)   # color_img in & color_img out
    
    # save image for debug
    # output_debug = np.append(masked_img,warp_img,1)
    # mpimg.imsave('./pipeline_outputs/3_prespectivetransform/'+str(cnt)+'.jpg', warp_img , cmap='gray')
    
    # you should return the final output (image with lines are drawn on lanes)
    # output = np.append(img,lane_img,1)
    
    mpimg.imsave('./pipeline_outputs/3_prespectivetransform/'+str(cnt)+'.jpg', output)
    return(output)


In [14]:
# Place holder for live processing

cnt = 0  # global counter
# images = glob.glob('./video2Image/projectVideoImage_*.jpg')
# images = glob.glob('./video2ImageSample/projectVideoImage_*.jpg')
images = glob.glob('./test_images/test*.jpg')


for image in sorted(images):
    print(image)
    img = mpimg.imread(image)
    out_img = process_image(img)
    

./test_images/test1.jpg
./test_images/test2.jpg
./test_images/test3.jpg
./test_images/test4.jpg
./test_images/test5.jpg
./test_images/test6.jpg


In [15]:
%load_ext autoreload
%autoreload 2
cnt = 0  # counter
output = 'output.mp4'
clip1 = VideoFileClip("project_video.mp4")
white_clip = clip1.fl_image(process_image) #NOTE: this function expects color images!!
%time white_clip.write_videofile(output, audio=False)

[MoviePy] >>>> Building video output.mp4
[MoviePy] Writing video output.mp4


100%|█████████▉| 1260/1261 [13:37<00:00,  1.66it/s]


[MoviePy] Done.
[MoviePy] >>>> Video ready: output.mp4 

CPU times: user 4min 49s, sys: 3.38 s, total: 4min 53s
Wall time: 13min 40s


In [16]:
HTML("""
<video width="960" height="540" controls>
  <source src="{0}">
</video>
""".format(output))